In [1]:
import json
import torch

In [9]:
flickr_data = json.load(open('eval_dataset/Flickr_Dataset.json'))
all_data = json.load(open('eval_dataset/FlickrCOCO_Dataset.json'))
data = flickr_data

In [10]:
def is_within_span(token_positives, span):
    assert len(span) == 2
    start_idx, end_idx = span[0], span[1]
    for span in token_positives:
        if start_idx >= span[0] and end_idx <= span[1]:
            return True
def find_dp_word_len(dpid):
    return len(data['grounding_pairs'][dpid]['data']['img_info']['caption'])
def find_phrase_num(dpid):
    return len(data['grounding_pairs'][dpid]['data']['noun_phrase_groups'])
def find_bbox_num(dpid):
    return len(data['grounding_pairs'][dpid]['data']['annotations'])
def find_bboxs_coverage_for_dpid_word(dpid, word):
    covs = []
    this_data = data['grounding_pairs'][dpid]['data']
    img_size = int(this_data['img_info']['width']) * int(this_data['img_info']['height'])
    word_span = None
    for group in this_data['noun_phrase_groups']:
        has = False
        for reg in group['region']:
            if reg[1] == word:
                has = True
                word_span = reg[3]
                break
        if has:
            for ann in this_data['annotations']:
                # this trivial implementation is buggy
                # if ann['bbox_info']['tokens_positive'] == spans:
                if is_within_span(ann['bbox_info']['tokens_positive'], word_span):
                    box = ann['bbox_info']['bbox']
                    b_s = box[-1] * box[-2]
                    cov = b_s / img_size
                    covs.append(cov)
    return covs
def find_bboxs_for_dpid_word(dpid, word):
    bboxs = []
    this_data = data['grounding_pairs'][dpid]['data']
    word_span = None
    for group in this_data['noun_phrase_groups']:
        has = False
        for reg in group['region']:
            if reg[1] == word:
                has = True
                word_span = reg[3]
                break
        if has:
            for ann in this_data['annotations']:
                if is_within_span(ann['bbox_info']['tokens_positive'], word_span):
                    bboxs.append(ann['bbox_info']['bbox'])
    return bboxs

In [11]:
word_list = set()
for t in data['tests']:
    word_list.add(t['mask_regions'][0][1])

In [12]:
from collections import defaultdict
word_dpid_map = defaultdict(list)

for dpid, t in enumerate(data['tests']):
    word_dpid_map[t['mask_regions'][0][1]].append(t['pair_id'])
def find_corresponded_dpids_for_words(K):
    return word_dpid_map[K]
def mean(l):
    return sum(l) / len(l)

In [13]:
from tqdm import tqdm
test_word_feats = {}

for word in tqdm(word_list):
    dps =find_corresponded_dpids_for_words(word)
    phrase_nums = [find_phrase_num(i) for i in dps]
    box_nums = [find_bbox_num(i) for i in dps]
    bbox_covs = [mean(find_bboxs_coverage_for_dpid_word(i, word)) for i in dps]
    test_word_feats[word] = {'word_freq':len(dps), 'avg_cooccur_phrase':mean(phrase_nums), 'avg_cooccur_box': mean(box_nums), 'avg_bbox_cov': mean(bbox_covs)}

100%|██████████| 4214/4214 [00:00<00:00, 11243.31it/s]


In [14]:
with open("all-test-word-feats-fixed.json", 'w') as f:
    json.dump(test_word_feats, f)